In [1]:
import ctypes
import json
import requests
import csmapi
import os

In [2]:
# The location of the usgscsm library:
lib ="$CONDA_PREFIX/lib/libusgscsm.so"
ctypes.CDLL(os.path.expandvars(lib))

<CDLL '/home/kberry/anaconda3/envs/csm/lib/libusgscsm.so', handle 555785378f00 at 0x7f23b66b4910>

In [3]:
label = """
PDS_VERSION_ID = PDS3                                                         
FILE_NAME = "J03_045994_1986_XN_18N282W.IMG"                                  
RECORD_TYPE = FIXED_LENGTH                                                    
RECORD_BYTES = 5056                                                           
FILE_RECORDS = 17409                                                          
LABEL_RECORDS = 1                                                             
^IMAGE = 2                                                                    
SPACECRAFT_NAME = MARS_RECONNAISSANCE_ORBITER                                 
INSTRUMENT_NAME = "CONTEXT CAMERA"                                            
INSTRUMENT_HOST_NAME = "MARS RECONNAISSANCE ORBITER"                          
MISSION_PHASE_NAME = "ESP"                                                    
TARGET_NAME = MARS                                                            
INSTRUMENT_ID = CTX                                                           
PRODUCER_ID = MRO_CTX_TEAM                                                    
DATA_SET_ID = "MRO-M-CTX-2-EDR-L0-V1.0"                                       
PRODUCT_CREATION_TIME = 2016-10-04T20:55:10                                   
SOFTWARE_NAME = "makepds05 $Revision: 1.16 $"                                 
UPLOAD_ID = "UNK"                                                             
ORIGINAL_PRODUCT_ID = "4A_04_10E802EF00"                                      
PRODUCT_ID = "J03_045994_1986_XN_18N282W"                                     
START_TIME = 2016-05-19T08:25:22.197                                          
STOP_TIME = 2016-05-19T08:25:54.867                                           
SPACECRAFT_CLOCK_START_COUNT = "1148113565:138"                               
SPACECRAFT_CLOCK_STOP_COUNT = "N/A"                                           
FOCAL_PLANE_TEMPERATURE = 296.0 <K>                                           
SAMPLE_BIT_MODE_ID = "SQROOT"                                                 
OFFSET_MODE_ID = "197/230/219"                                                
LINE_EXPOSURE_DURATION = 1.877 <MSEC>                                         
SAMPLING_FACTOR = 1                                                           
SAMPLE_FIRST_PIXEL = 0                                                        
RATIONALE_DESC = "Ride-along with HiRISE"                                     
DATA_QUALITY_DESC = "OK"                                                      
ORBIT_NUMBER = 45994                                                          
OBJECT = IMAGE                                                                
LINES = 17408                                                                 
LINE_SAMPLES = 5056                                                           
LINE_PREFIX_BYTES = 0                                                         
LINE_SUFFIX_BYTES = 0                                                         
SAMPLE_TYPE = UNSIGNED_INTEGER                                                
SAMPLE_BITS = 8                                                               
SAMPLE_BIT_MASK = 2#11111111#                                                 
CHECKSUM = 16#B79E51AD#                                                       
END_OBJECT = IMAGE                                                            
END    
"""

In [4]:
url = "https://astroservices.usgs.gov/pfeffer/v1/pds/"
resp = requests.post(url, json={"label":label})

# File to output the returned ISD information to: 
isd_file = '/scratch/csm2020/J03_045994_1986_XN_18N282W.json'
with open(isd_file, 'w') as f:
    d = resp.json()
    json.dump(d, f)
    

In [5]:
plugin = csmapi.Plugin.findPlugin('UsgsAstroPluginCSM')
isd = csmapi.Isd(isd_file)
model_name = resp.json()['name_model']

In [6]:
if plugin.canModelBeConstructedFromISD(isd, model_name):
    model = plugin.constructModelFromISD(isd, model_name)

In [7]:
isize = model.getImageSize()
isize.line, isize.samp

(17408.0, 5056.0)

In [8]:
ic = csmapi.ImageCoord(100, 100)
gnd = model.imageToGround(ic, 100)
gnd.x, gnd.y, gnd.z

(710344.1805031475, 3157701.033023591, 1023078.7673066386)